# Start

Solve this Kaggle problem https://www.kaggle.com/competitions/job-recommendation/overview

In [20]:
import pandas as pd
import gcsfs

# Start Loading Data

### Load the applications data

In [21]:
#applications_df = spark.read.format("csv").option("header", "true").option("delimiter", "\t").load("/Users/ganapathychidambaram/Desktop/predixions/apps.tsv")

gcs_path = 'gs://kaggle-competition-jobs/apps.tsv'
applications_df = pd.read_csv(gcs_path, sep='\t')

In [22]:
applications_df

,UserID,WindowID,Split,ApplicationDate,JobID
0,47,1,Train,2012-04-04 15:56:23.537,169528
1,47,1,Train,2012-04-06 01:03:00.003,284009
2,47,1,Train,2012-04-05 02:40:27.753,2121
3,47,1,Train,2012-04-05 02:37:02.673,848187
4,47,1,Train,2012-04-05 22:44:06.653,733748
...,...,...,...,...,...
1603106,1472089,7,Train,2012-06-23 16:06:03.087,573732
1603107,1472089,7,Train,2012-06-25 23:20:35.603,39401
1603108,1472089,7,Train,2012-04-30 14:01:42.52,175198
1603109,1472089,7,Train,2012-04-30 13:52:45.823,1073263


In [23]:
applications_df.dtypes

UserID              int64
WindowID            int64
Split              object
ApplicationDate    object
JobID               int64
dtype: object

pre-processing = Dropping null values, and again checking the count

In [24]:
applications_df = applications_df.dropna()

In [25]:
applications_df.describe()

,UserID,WindowID,JobID
count,1.603111e+06,1.603111e+06,1.603111e+06
mean,7.627023e+05,3.643567e+00,5.652615e+05
std,4.225649e+05,2.029082e+00,3.239136e+05
min,7.000000e+00,1.000000e+00,1.000000e+00
25%,3.936690e+05,2.000000e+00,2.805265e+05
50%,7.841790e+05,4.000000e+00,5.678070e+05
75%,1.126113e+06,5.000000e+00,8.498670e+05
max,1.472095e+06,7.000000e+00,1.116313e+06


The count remains the same after dropping null values, meaning there were no null values.

In [26]:
!pip3 install scikit-learn
!pip3 install xgboost
!pip3 install catboost

In [27]:
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [28]:
# Convert 'ApplicationDate' to datetime
applications_df['ApplicationDate'] = pd.to_datetime(applications_df['ApplicationDate'],format='mixed')

# Convert 'JobID' to numeric
applications_df['JobID'] = pd.to_numeric(applications_df['JobID'])

# Feature Engineering (add more features if needed)
applications_df['DayOfWeek'] = applications_df['ApplicationDate'].dt.dayofweek
applications_df['HourOfDay'] = applications_df['ApplicationDate'].dt.hour

In [29]:
unique_days = applications_df['DayOfWeek'].unique()
unique_days

array([2, 4, 3, 0, 5, 1, 6], dtype=int32)

In [30]:
unique_hours = applications_df['HourOfDay'].unique()
unique_hours

array([15,  1,  2, 22, 17, 20, 10, 21, 14,  9, 13, 18,  8,  7, 11,  5,  4,
        0, 12, 23, 19, 16,  3,  6], dtype=int32)

In [31]:
null_counts = applications_df.isnull().sum()

# Print the null counts for each column
print("Number of null values in each column:")
print(null_counts)

Number of null values in each column:
UserID             0
WindowID           0
Split              0
ApplicationDate    0
JobID              0
DayOfWeek          0
HourOfDay          0
dtype: int64


In [32]:
train_data = applications_df[applications_df['Split'] == 'Train']
test_data = applications_df[applications_df['Split'] == 'Test']

In [33]:
# Features and target variable
X_train = train_data[['UserID', 'WindowID', 'DayOfWeek', 'HourOfDay']]
y_train = train_data['JobID']

In [34]:
X_train.shape

(1417514, 4)

In [35]:
y_train.shape

(1417514,)

In [36]:
y_train.dropna().shape

(1417514,)

In [37]:

X_train = X_train.head(10000)
y_train = y_train.head(10000)


In [38]:
# Features and target variable
X_test = test_data[['UserID', 'WindowID', 'DayOfWeek', 'HourOfDay']]
y_test = test_data['JobID']

In [ ]:
# Train the model (using RandomForestClassifier as an example)
# model = xgb.XGBClassifier()
from catboost import CatBoostClassifier

model = CatBoostClassifier(iterations=1000, depth=6, learning_rate=0.1, verbose=100)
model.fit(X_train, y_train)

model.fit(X_train, y_train)

0:	learn: 8.9973447	total: 40m 37s	remaining: 28d 4h 25m 26s


In [ ]:
# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)